# Untitled Supermarked Project
Goal of the project is to gain insights on retail behaviour combining Data Science and Queuing Theory.

## Setting up Workshop

### Importing Libraries

In [1]:
# DataFrames
import pandas as pd

## 01 : Import Data

Source: https://www.kaggle.com/agasca/retail-sales/data

In [2]:
data_raw = pd.read_csv(filepath_or_buffer = "Sales.csv")

## 02 : Clean Data
|Input|Tools & Techniques|Output|
|-|-|-|
|Raw Data|Select Features|Clean Data|
||Rename Features||
||Handle Feature Format||


The dataset has no missing values.

### 02-01 : Select Features
The Dataset consists of 5 features: 'Folio', 'Hora', 'Total', 'Pago', and 'Cajero'. The first one bears no usefull information and will therefore be dropped.

In [23]:
data_cln = data_raw.drop('Folio', axis = 1)

### 02-02 : Rename Features
The 4 remaining Features will be translated for transparancy.

In [24]:
data_cln.columns = ['time','price','paid','cashier']

### 02-03 : Handle Feature Format
Turning the 'time' feature into timestamp.

In [25]:
data_cln.time = pd.to_datetime(data_cln.time, format='%Y-%m-%d %H:%M:%S')

## 03 : Data Analysis
|Input|Tools & Techniques|Output|
|-|-|-|
|Clean Data|Single Feature Analysis|Data Understanding|
||Multiple Feature Analysis|Business Understanding|

### 03-01 : Single Feature Analysis

#### 03-01-01 : Time

In [37]:
print("Dataset spans from",
      data_cln.time.min(),
      "to",data_cln.time.max(),
      ". A timeperiod of",
      data_cln.time.max()-data_cln.time.min(),
      ".")

Dataset spans from 2018-05-01 07:46:22 to 2018-08-31 21:13:28 . A timeperiod of 122 days 13:27:06 .


#### 03-01-02 : Price

#### 03-01-02 : Paid

#### 03-01-03 : Cashier

### 03-02 : Multiple Feature Analysis

In [ ]:
# Correlations

In [ ]:
# Time vs Paid

In [ ]:
# Time vs Price

In [ ]:
# Time vs Cashier

In [ ]:
# Paid vs Price

In [ ]:
# Paid vs Cashier

In [ ]:
# Comment: Effects of Daylight Savingstime

In [ ]:
# Price vs Cashier

## Future Works

- Reformat Data to "cashier shift" by combining groupby() and resample(), try those "\".<br>
- Finish Analysis<br>
- Come up with Supermarket related pun as project title<br>
- Consider Visualizations via Bookeh<br>